In [1]:
import plotly
from plotly.graph_objs import Scatter,Layout
from plotly.offline import plot

import requests
import pandas as pd
import time
import os
from io import StringIO

def truncate(sio):
    sio.truncate(0)
    sio.seek(0)
    return sio

class stock:
    def __init__(self,save_file_path):
        if not os.path.exists(save_file_path):
            os.makedirs(save_file_path)
        pass

    def run(self,stocks_list,months_range,save_file_path):
        for stock in stocks_list: 
            columns = ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
            my_df = pd.DataFrame(columns = columns)
            for m in months_range: 
                url = 'https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date='+ m +'&stockNo='+stock
                print('processing:',url)
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
                res = requests.get(url,headers = headers)
                io_csv_info = StringIO(res.text)
                first_line=io_csv_info.readline()
                io_csv_info.readline() 

                while True: 
                    line = io_csv_info.readline()
                    info_list = line.split('","')
                    info_list[0]  = info_list[0].strip('"')
                    info_list[-1] = info_list[-1].strip('",\r\n')
                    
                    if '"說明:"\r\n' in line :
                        break

                    df = pd.DataFrame([info_list], columns = columns) 
                    my_df = my_df.append(df , ignore_index = True) 
                truncate(io_csv_info) 
                time.sleep(7)
            my_df.insert(1,"股票代碼",first_line[9:13],True)
            my_df.insert(2,"股票名稱",first_line[14:20],True)    
            my_df.to_csv(save_file_path + f'/{stock}_from_{start_month}_to_{end_month}.csv' , encoding = 'big5')
            print(f'=====================finished {first_line[9:20]} crawl========================================')
            """
            作圖
            data=[
                    Scatter(x=my_df["日期"],y=my_df['收盤價'],name='收盤價'),
                    Scatter(x=my_df["日期"],y=my_df['最低價'],name='最低價'),
                    Scatter(x=my_df["日期"],y=my_df['最高價'],name='最高價')
                    ]
            plot({"data":data,"layout":Layout(title='個股統計圖')},auto_open=True)
            """
            
if __name__ == '__main__':          
    stocks_list = str(input('stocks list (ex:2317,2330,2303)')).split(',')
    start_month = str(input('month start (ex:2020-01):'))
    end_month   = str(input('month end   (ex:2022-02):'))
    save_file_path = str(input('save file path(ex: ./csv_data)'))

    months_range = [str(date).replace('-','').split(' ')[0] for date in pd.date_range(start=start_month ,end = end_month,freq = 'MS')]
    
    S=stock(save_file_path) 
    S.run(stocks_list,months_range,save_file_path) 



stocks list (ex:2317,2330,2303)
month start (ex:2020-01):
month end   (ex:2022-02):
save file path(ex: ./csv_data)


ValueError: Neither `start` nor `end` can be NaT